## Startup

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import aic, bic

import os
import pickle

In [14]:
np.random.seed(42)

In [15]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

## Data Retrieval

In [16]:
start='2013-01-01'
end="2023-06-01"

name=f'processed_dataset_{start}_{end}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    data=pickle.load(handle)
    
name=f'finaldf_dataset_{start}_{end}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)

In [17]:
df.head(3)

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,YPF_rets,...,BMA_gk_vol,BMA.BA_rets,BMA.BA_log_rets,BMA.BA_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-03,0.007552,0.007524,0.000129,-0.012748,-0.012830,0.001228,0.010616,0.010560,0.000677,-0.004016,...,0.000557,-0.007812,-0.007843,0.000857,0.011942,0.011871,0.000020,-0.002186,-0.002188,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.010043,-0.010094,0.000554,-0.006303,-0.006323,0.000208,-0.000672,...,0.000695,-0.027559,-0.027946,0.000692,0.021496,0.021269,0.000228,-0.008133,-0.008167,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,-0.014493,-0.014599,0.000517,0.002114,0.002112,0.000063,0.005380,...,0.000862,0.012146,0.012073,0.000494,-0.004061,-0.004069,0.000017,-0.007749,-0.007779,0.000022


# VAR Training

In [18]:
log_rets_list=[column for column in df.columns if column.endswith("log_rets")]
vol_list=[column for column in df.columns if column.endswith("vol")]
simple_rets_list=[column for column in df.columns if (column.endswith("log_rets")) and (column not in log_rets_list)]

In [76]:
def is_best_aic(results, lag):
    """
    Esta función revisa si el aic del modelo es mejor
    """
    best_aic=results_dict[stock]['best_aic']
    aic_score=results.aic

    if aic_score < best_aic:
        best_aic = aic_score
        best_lag_aic = lag

        results_dict[stock]['best_aic']= best_aic 
        results_dict[stock]['best_lag_aic']= best_lag_aic
        model_dict[stock]["aic"]=results
    
    else:
        pass

In [77]:
def is_best_bic(results, lag):
    """
    Esta función revisa si el bic del modelo es mejor
    """
    best_bic=results_dict[stock]['best_bic']
    bic_score=results.bic

    if bic_score < best_bic:
        best_bic = bic_score
        best_lag_bic = lag

        results_dict[stock]['best_bic']= best_bic 
        results_dict[stock]['best_lag_bic']= best_lag_bic
        model_dict[stock]["bic"]=results

    else:
        pass

In [78]:
init_dict={"best_aic":np.inf,
           "best_lag_aic":None,
           "best_bic": np.inf,
           "best_lag_bic" : None}
results_dict={stock:init_dict for stock in tickerlist}

model_dict={stock:{"aic":None, "bic":None} for stock in tickerlist}

In [79]:
for stock in tickerlist:
    columns = ['USD_log_rets', 'USD_gk_vol', f'{stock}_log_rets', f'{stock}_gk_vol']
    stock_data = df[columns]
    
    for lag in range(1, 11):
        model = VAR(stock_data)
        results = model.fit(lag)

        is_best_aic(results, lag)
        is_best_bic(results, lag)
        
        
# Print the results
for stock, results in results_dict.items():
    print(f"Stock: {stock}, Best lag based on AIC: {results['best_lag_aic']}, AIC: {results['best_aic']}")


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so

Stock: ^MERV, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: GGAL, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: GGAL.BA, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: YPF, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: YPFD.BA, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: EDN, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: EDN.BA, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: BMA, Best lag based on AIC: 8, AIC: -42.92942685502185
Stock: BMA.BA, Best lag based on AIC: 8, AIC: -42.92942685502185


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so

In [ ]:
with open(os.path.join(resultsroute, "VARdict.pickle"), "wb") as f:
    pickle.dump(var_models, f, pickle.HIGHEST_PROTOCOL)